In [1]:
from pathlib import Path

import pandas as pd

import numpy as np

import meam2nn

Define function for comparing LAMMPS parameter files

In [2]:
def compare_parameters(params1, params2, ignore_missing=None, ignore_different=None, rtol=1e-05, atol=1e-08):
    """
    Compares MEAM parameters from two parameter files and identifies the parameters that
    are missing from one or the other files and the paramters that have different values.
    
    Parameters
    ----------
    params1 : dict
        The parameter file parameters from one of the files to compare.
    params2 : dict
        The parameter file parameters from one of the files to compare.
    ignore_missing : list, optional
        Parameter keys to ignore if missing from one of the files.
    ignore_different : list, optional
        Parameter keys to ignore if the value is different between the files.
    rtol : float, optional
        The relative tolerance to use in comparing numerical values.  Default value is 1e-5.
    atol : float, optional
        The absolute tolerance to use in comparing numerical values.  Default value is 1e-8.
    """
    
    missing1 = []
    missing2 = []
    different = []
    if ignore_missing is None:
        ignore_missing = []
    if ignore_different is None:
        ignore_different = []
    
    for key in params1:
        if key not in params2:
            if key not in ignore_missing:
                missing2.append(key)
        else:
            try:
                if not np.isclose(params1[key], params2[key], rtol=rtol, atol=atol) and key not in ignore_different:
                    different.append(key)
            except:
                if params1[key] != params2[key]:
                    different.append(key)                
        
    for key in params2:
        if key not in params1:
            if key not in ignore_missing:
                missing1.append(key)
    
    if len(missing1) > 0:
        print('First parameter set is missing:')
        for key in missing1:
            print('-', key)
    
    if len(missing2) > 0:
        print('Second parameter set is missing:')
        for key in missing2:
            print('-', key)
            
    if len(different) > 0:
        print('Different values are:')
        for key in different:
            print('-', key)
            print('  *', params1[key])
            print('  *', params2[key])
            
    if len(missing1) == 0 and len(missing2) == 0 and len(different) == 0:
        print('All values match!')

Search for LAMMPS files

In [23]:
library_files = []
parameter_files = []

for fname in Path('LAMMPS').glob('*/*/*.meam'):
    if fname.name == 'library.meam':
        library_files.append(fname)
    else:
        parameter_files.append(fname)

Compare parameter file values between csv files and LAMMPS \*.meam files 

In [29]:
db = meam2nn.MEAM2NN()

ignore_missing = ['mixture_ref_t', 'emb_lin_neg', 'bkgd_dyn', 'alpha(1,1)', 'alpha(2,2)', 'alpha(3,3)',
                  'Cmin(2,1,3)', 'Cmin(3,1,2)', 'Cmin(3,2,1)', 'Cmax(2,1,3)', 'Cmax(3,1,2)', 'Cmax(3,2,1)']
ignore_missing_element = ['ialloy', 'rho0(1)']
ignore_different = ['rc']

for paramfile in parameter_files:
    
    # Extract names of elements and order from the file name
    elementstr = paramfile.stem.replace('-para', '')
    uppers = []
    for i in range(len(elementstr)):
        if elementstr[i].isupper():
            uppers.append(i)
    elements = []
    for i in range(len(uppers)):
        s = uppers[i]
        try:
            e = uppers[i+1]
        except:
            elements.append(elementstr[s:])
        else:
            elements.append(elementstr[s:e])

    print(paramfile)
    params2 = meam2nn.parse_lammps_parameter_file(paramfile)
    
    # Identify all symbol models associated with the elements
    allsymbols = []
    if len(elements) == 1:
        matches = db.elements[db.elements.El == elements[0]]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym])
    elif len(elements) == 2:
        matches = db.alloys[((db.alloys.El1==elements[0]) & (db.alloys.El2==elements[1]))]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym1, matches.loc[index].Sym2])
    elif len(elements) == 3:
        matches = db.ternaries[((db.ternaries.El1==elements[0]) & (db.ternaries.El2==elements[1]) & (db.ternaries.El3==elements[2]))]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym1, matches.loc[index].Sym2, matches.loc[index].Sym3])
    else:
        print('Skipping >3 elements for now...')
    
    if len(allsymbols) == 0:
        print('No matching parameters found!')
        
    # Loop over all matching symbols and compare
    for symbols in allsymbols:
        try:
            params1 = meam2nn.parse_lammps_parameter_file(db.lammps_parameter(symbols))
        except:
            pass
        else:
            print(symbols)
            compare_parameters(params1, params2, ignore_missing=ignore_missing, ignore_different=ignore_different)
    print()

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Cr\1\Cr.meam
['Cr']
All values match!
['CR']
Different values are:
- repuls(1,1)
  * 0.1
  * 0.0
- attrac(1,1)
  * 0.02
  * 0.0
- Cmin(1,1,1)
  * 0.71
  * 0.78

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Fe\1\Fe.meam
['Fe']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Mo\1\Mo.meam
['Mo']
All values match!
['MO']
Different values are:
- Cmin(1,1,1)
  * 0.82
  * 0.64
- Cmax(1,1,1)
  * 2.5
  * 2.8

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Nb\1\Nb.meam
['Nb']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Ta\1\Ta.meam
['Ta']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--V\1\V.meam
['V']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--W\1\W.meam
['W']
All values match!

LAMMPS\2001--Lee-B-J-Shim-J-H-Park-H-M--Fe-Cr\1\FeCr.meam
['Fe', 'Cr']
All values match!

LAMMPS\2003--Lee-B-J-Shim-J-H-Baskes-M-I--Ag\1\Ag.meam
['Ag']
All values match!

LAMMPS\2003--Lee-B-J

['Pd', 'Al']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Co\1\PdCo.meam
['Pd', 'Co']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Cu\1\PdCu.meam
['Pd', 'Cu']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Fe\1\PdFe.meam
['Pd', 'Fe']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Mo\1\PdMo.meam
['Pd', 'MO']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Ni\1\PdNi.meam
['Pd', 'Ni']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Ti\1\PdTi.meam
['Pd', 'Ti']
All values match!

LAMMPS\2019--Jang-H-S-Seol-D-Lee-B-J--Mg-Zn-Ca\1\CaZnMg.meam
['Ca', 'Zn', 'Mg']
All values match!

LAMMPS\2020--Jang-H-S-Seol-D-Lee-B-J--Al-Ca\1\AlCa.meam
No matching parameters found!

LAMMPS\2020--Jeong-G-U-Lee-B-J--Pd-C\1\PdC.meam
['Pd', 'C']
All values match!

LAMMPS\2020--Jeong-G-U-Lee-B-J--Pt-C\1\PtC.meam
['Pt', 'C']
All values match!

LAMMPS\2020--Oh-S-H-Seol-D-Lee-B-J--Co-Ti\1\CoTi

In [4]:
interactions

,interaction,doi,parameter file LAMMPS,library file LAMMPS,parameter file csme,library file csme,known issues
0,Cr,10.1103/physrevb.64.184102,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Cr/1/C...,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Cr/1/l...,"29 Potential parameter files for Fe, Cr, Mo, W...","29 Potential parameter files for Fe, Cr, Mo, W...",NaN
1,Fe,10.1103/physrevb.64.184102,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Fe/1/F...,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Fe/1/l...,"29 Potential parameter files for Fe, Cr, Mo, W...","29 Potential parameter files for Fe, Cr, Mo, W...",NaN
2,Mo,10.1103/physrevb.64.184102,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Mo/1/M...,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Mo/1/l...,"29 Potential parameter files for Fe, Cr, Mo, W...","29 Potential parameter files for Fe, Cr, Mo, W...",NaN
3,Nb,10.1103/physrevb.64.184102,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Nb/1/N...,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Nb/1/l...,"29 Potential parameter files for Fe, Cr, Mo, W...","29 Potential parameter files for Fe, Cr, Mo, W...",NaN
4,Ta,10.1103/physrevb.64.184102,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Ta/1/T...,2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Ta/1/l...,"29 Potential parameter files for Fe, Cr, Mo, W...","29 Potential parameter files for Fe, Cr, Mo, W...",NaN
...,...,...,...,...,...,...,...
127,Co-Cu,10.1557/jmr.2002.0135,NaN,NaN,NaN,NaN,NaN
128,Na,10.1016/j.commatsci.2020.109953,NaN,NaN,NaN,NaN,NaN
129,Na-Sn,10.1016/j.commatsci.2020.109953,NaN,NaN,NaN,NaN,NaN
130,Al-Ca-Mg,NaN,NaN,NaN,57 Potential parameter files for Mg-Al-Ca and ...,57 Potential parameter files for Mg-Al-Ca and ...,NaN


In [47]:
location = 'lammps'
#location = 'csme'

db = meam2nn.MEAM2NN()
interactions = pd.read_csv('interactions.csv')

ignore_missing = ['mixture_ref_t', 'emb_lin_neg', 'bkgd_dyn', 'alpha(1,1)', 'alpha(2,2)', 'alpha(3,3)',
                  'Cmin(2,1,3)', 'Cmin(3,1,2)', 'Cmin(3,2,1)', 'Cmax(2,1,3)', 'Cmax(3,1,2)', 'Cmax(3,2,1)']
ignore_element = ['ialloy', 'rho0(1)']
ignore_different = ['rc']

if location == 'lammps': 
    root = Path('LAMMPS')
    paramkey = 'parameter file LAMMPS'
elif location == 'csme':
    root = Path('cmse_postech_ac_kr')
    paramkey = 'parameter file csme'

for i in interactions.index:
    row = interactions.loc[i]
    symbols = row.interaction.split('-')
    print(row.interaction)
    
    try:
        params1 = meam2nn.parse_lammps_parameter_file(db.lammps_parameter(symbols))
    except:
        print('No matching database parameters found')
        print()
        continue
    
    try:
        paramfile = Path(root, row[paramkey])
        assert paramfile.is_file()
    except:
        print('No parameter file found')
        print()
        continue        
    else:
        params2 = meam2nn.parse_lammps_parameter_file(paramfile)
            
    if len(symbols) == 1:
        compare_parameters(params1, params2, ignore_missing=ignore_missing+ignore_element,
                           ignore_different=ignore_different+ignore_element)
        print()
    else:
        compare_parameters(params1, params2, ignore_missing=ignore_missing, ignore_different=ignore_different)
        print()

Cr
All values match!

Fe
All values match!

Mo
All values match!

Nb
All values match!

Ta
All values match!

V
All values match!

W
All values match!

Fe-Cr
All values match!

Ag
All values match!

Al
All values match!

Au
All values match!

Cu
All values match!

Ni
All values match!

Pb
All values match!

Pd
All values match!

Pt
All values match!

Ni-W
All values match!

Ni-Cu
All values match!

C
All values match!

Fe-Cu
All values match!

Pt-Fe
All values match!

Ti
All values match!

Zr
All values match!

Fe-C
All values match!

Fe-N
All values match!

Ti-Cu
All values match!

Fe-H
All values match!

Si
All values match!

Al-Ni
All values match!

In
All values match!

Ge
All values match!

Zr-Cu
All values match!

Ti-C
All values match!

Ti-N
All values match!

Nb-Fe
All values match!

Fe-Ti
All values match!

Ga-In
Different values are:
- Ec(1,2)
  * 2.8405
  * 4.06
- alpha(1,2)
  * 4.63256713599982
  * 3.87486077
- attrac(1,2)
  * 0.0735
  * 0.074
- repuls(1,2)
  * 0.0735
  * 0

In [40]:
db = meam2nn.MEAM2NN()
db.ternary(['Ni', 'Al', 'Co'])

El1                                    Ni
El2                                    Al
El3                                    Co
Sym1                                   Ni
Sym2                                   Al
Sym3                                   Co
DOI         10.1088/0965-0393/23/5/055004
Cmax_ikj                             1.44
Cmax_ijk                             2.38
Cmax_jik                             1.44
Cmin_ikj                             0.56
Cmin_ijk                             0.97
Cmin_jik                             0.56
Rcut                                  4.5
AU                                    NaN
Name: 10, dtype: object

In [41]:
db = meam2nn.MEAM2NN()
db.alloy(['Al', 'Co'])

El1                                    Al
El2                                    Co
Sym1                                   Al
Sym2                                   Co
DOI         10.1016/j.calphad.2012.04.001
Ref.St                             BCC_B2
delta_Ec                           -0.565
Re                                 2.4768
B                                  1.0112
B*                                1.62012
alpha                             4.89086
d+                                  0.025
d-                                  0.025
Cmin_iji                              1.1
Cmin_jij                             0.49
Cmin_iij                             0.49
Cmin_ijj                             0.49
Cmax_iji                              2.8
Cmax_jij                                2
Cmax_iij                              2.8
Cmax_ijj                             1.44
Rcut                                  4.5
Au                                     DW
lat                               

In [10]:
row

interaction                                                           Si-O
doi                                             10.1103/physrevb.93.144110
parameter file LAMMPS                                                  NaN
library file LAMMPS                                                    NaN
parameter file csme      37 Potential parameter files for Si-O system/l...
library file csme        37 Potential parameter files for Si-O system/l...
known issues                                                           NaN
Name: 116, dtype: object

In [43]:
E_Ga = 2.9
E_In = 2.51
E_NN = 4.88

dE_GaIn = -.1355
dE_GaNN = 0.56
dE_InNN = 0.1650

print(0.5 * E_Ga + 0.5* E_In - dE_GaIn)
print(0.5 * E_Ga + 0.5* E_NN - dE_GaNN)
print(0.5 * E_In + 0.5* E_NN - dE_InNN)

2.8405
3.3299999999999996
3.53


In [44]:
0.5 * (E_Ga + E_In) + 0.1355

2.8405

In [45]:
0.5 * (E_Ga + E_NN) - 0.56

3.3299999999999996

In [46]:
0.5 * (E_In + E_NN) - 0.1650

3.53